# pyradiance_walkthrough

## Install requirements in Google Collab

In [ ]:
!git clone -b development https://github.com/shirubana/pyrad_workshop_24.git

In [ ]:
cd pyrad_workshop_24

In [ ]:
!pip install requirements.txt

In [5]:
import datetime

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pyradiance as pr

### Make a scene

In [ ]:
my_scene = pr.Scene(
    "my_scene",
    surfaces=["Objects/room.rad"],
    materials=["Objects/materials.mat"])

my_view = pr.View(
    vtype='a',
    position=(6, 6, 4),
    direction=(0, -1, 0),
    horiz=180,
    vert=180,
)
my_scene.add_view(my_view)

dt = datetime.datetime.now()
for prim in pr.parse_primitive(pr.gendaylit(dt, 37, 122, 120, dirnorm=800, diffhor=80).decode()):
    my_scene.add_source(prim)
my_scene.add_source(pr.Primitive("skyfunc", "glow", "skyglow", [], [1, 1, 1, 0]))
my_scene.add_source(pr.Primitive("skyglow", "source", "skydome", [], [0, 0, 1, 180]))
my_scene.add_source(pr.Primitive("skyglow", "source", "groundplane", [], [0, 0, -1, 180]))

### Render

In [ ]:
img = pr.render(my_scene, resolution=(800, 800), ambbounce=1, nproc=8, quality="L")

### Tone-mapping

In [ ]:
with open("raw.hdr", "wb") as f:
    f.write(img)
img_tm = pr.pcond("raw.hdr", human=True)
_, yres, _, xres = pr.getinfo(img, dimension_only=True).decode().split()
xres, yres = int(xres), int(yres)
img_data = pr.pvalue(img_tm, header=False, outform='f', resstr=False)
img_array = np.frombuffer(img_data, dtype=np.single).reshape(xres, yres, 3)
plt.imshow(img_array)

### Luminance falsecoloring

In [ ]:
img_data = pr.pvalue(img, header=False, outform='f', resstr=False)
img_array = np.frombuffer(img_data, dtype=np.single).reshape(xres, yres, 3)
luminance = img_array[:, :, 0] * 47.4 + img_array[:, :, 1] * 119.9 + img_array[:, :, 2] * 11.6
plt.imshow(luminance, cmap='viridis', norm=mpl.colors.LogNorm())
plt.colorbar()

### Set sampling parameters

In [ ]:
sp = pr.SamplingParameters()
sp.aa = 0
sp.ad = 32
sp.ab = 2

### Rendering with rtpict

In [ ]:
img = pr.rtpict(my_view, my_scene.octree, xres=3200, yres=3200, nproc=8, params=sp.args())

### Tone-mapping

In [ ]:
pwd

In [ ]:
with open("raw2.hdr", "wb") as f:
    f.write(pr.pfilt(img, one_pass=True, xres="/4", yres="/4", gaussian_filter_radius=0.6))
img_arr = np.frombuffer(pr.pvalue(pr.pcond("raw2.hdr", human=True), header=False, outform='f', resstr=False), dtype=np.single).reshape(800, 800, 3)

In [ ]:
plt.imshow(img_arr)

### BSDF analysis

In [ ]:
ttree = pr.BSDF("./ms6858_g6.xml")

In [ ]:
ttree.info
ttree.components

In [ ]:
klems = pr.BSDF("./ms6858_kf.xml")
klems.info
klems.components

In [ ]:
ttree_tau_h = []
klems_tau_h = []
for i in range(90):
    ttree_tau_h.append(ttree.direct_hemi(i, 0, 't'))
    klems_tau_h.append(klems.direct_hemi(i, 0, 't'))

In [ ]:
plt.plot(ttree_tau_h, label="ttree")
plt.plot(klems_tau_h, label="klems")
plt.xlabel("theta (deg)")
plt.ylabel("tau_h")
plt.legend()

### Generate spectral sky primitives

In [ ]:
ssky = pr.genssky(
    datetime.datetime.now(),
    latitude=40.8,
    longitude=111.9,
    timezone=105,
    nthreads=8,
)

In [ ]:
sky_glow = str(pr.Primitive("skyfunc", "glow", "skyglow", [], [1,1,1,0])).encode()
sky_source = str(pr.Primitive("skyglow", "source", "skydome", [], [0,0,1,180])).encode()
ground_source = str(pr.Primitive("skyglow", "source", "groundplane", [], [0,0,-1,180])).encode()

### Modify this to generate the octree, right now it only contains the skys

In [ ]:
with open("sky.oct", "wb") as f: f.write(pr.oconv(stdin=ssky+sky_glow+sky_source+ground_source))

### Render an image